# NYC Taxi dataset - anomaly detection with Autoencoder

### Libraries and settings

In [ ]:
# libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, Dropout, TimeDistributed

import warnings

: 

In [ ]:
# settings
plt.rcParams['figure.figsize'] = (10, 4)
sns.set()
warnings.filterwarnings('ignore')

### Dataset

```nyc_taxi.csv``` - number of NYC taxi passengers, where the five anomalies occur during the NYC marathon, Thanksgiving, Christmas, New Years Day, and a snow storm. The raw data is from the [NYC Taxi and Limousine Commission](https://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). The data consists of aggregating the total number of taxi passengers into 30 minute buckets.

![info](https://raw.githubusercontent.com/bartk97/NYC-Taxi-Anomaly-Detection/main/Images/Data%20with%20highlighted%20anomalies.png)

### Goal
I am going to detect anomalies in the dataset.

### Import data
Importing data from https://github.com/numenta/NAB

In [ ]:
url = 'https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/nyc_taxi.csv'
data = pd.read_csv(url, parse_dates=['timestamp'], index_col='timestamp')
data.head()

We can see that the data is split into 30 minute buckets so there are $24\cdot2 = 48$ data points per day.

In [ ]:
period = 24 * 2
data.index

### Some information about the data

In [ ]:
print('From  ' + str(np.min(data.index)) + '  to  ' +str(np.max(data.index)))

In [ ]:
print('Data size: %d \nNumber of data per day: %d \nNumber of days: %d' %(data.shape[0], period, data.shape[0] / period))

In [ ]:
print('Missing value: ', data.isnull().to_numpy().sum())

### Time Series Plot

In [ ]:
data.plot(title='Data points', figsize=(30,4))
plt.show()

### Plots with the data from first 5 days

In [ ]:
for i in range(5):
    data[period*i:period*(i + 1)].plot(figsize=(5, 2), title=str(data.index[period * i])[:10])
    plt.xlabel(None)
    plt.show()

### Splitting data into days

I split a time series into days and created a new DataFrame as follows: one row corresponds to one day and one column corresponds to 30-minute interval.

In [ ]:
# splitting data into days - function
def create_dataset(X, dates, period=1):
    Xs = []
    indexes = []
    for i in range(int(len(X) / period)):
        v = X.iloc[i*period: (i + 1)*period].values
        indexes.append(dates[period*i])
        Xs.append(v)        
    return np.array(Xs), np.array(indexes)

In [ ]:
# new data frame
df, dates = create_dataset(data.value, data.index, period)
df = pd.DataFrame(df, dates)

print('df.shape: ', df.shape)
df.head()

### Mean and standard deviation function (w.r.t. hours)

In [ ]:
plt.plot(df.mean(), label='mean')
plt.fill_between(df.columns, df.mean() + df.std(), df.mean() - df.std(), alpha=0.3, label='$\pm$ std')
plt.fill_between(df.columns, df.mean() + 2*df.std(), df.mean() - 2*df.std(), alpha=0.1, label= '$\pm$ 2std')
plt.legend()
plt.show()

### Mean and standard deviation function vs Data Points

In [ ]:
n_days = 15
mean_fucntion = np.tile(df.mean(), n_days)
std_function = np.tile(df.std(), n_days)

plt.figure(figsize=(30, 4))
plt.plot(mean_fucntion, label='mean')
plt.fill_between(np.arange(period*n_days), mean_fucntion - std_function, mean_fucntion + std_function, alpha=0.2, label='std')
plt.plot(data.to_numpy().flatten()[:period*n_days], label='data')
plt.legend()
plt.show()

### Splitting data into train and test sets

In [ ]:
# splitting
ratio = 0.55
train_size = int(df.shape[0] * ratio)
X_train = df[:train_size]
X_test = df[train_size:]

dates_train = np.array(df.index[:train_size], dtype='datetime64[D]')
dates_test = np.array(df.index[train_size:], dtype='datetime64[D]')


# info
print('Train size: ', ratio)
print('\n\nTRAIN SET:  from  ' + str(np.min(dates_train)) + '  to  ' +str(np.max(dates_train)))
print('Data size: ', X_train.shape[0])
print('Number of days: ', int(X_train.shape[0] / period))
print('\n\nTEST SET:  from  ' + str(np.min(dates_test)) + '  to  ' +str(np.max(dates_test)))
print('Data size: ', X_test.shape[0])
print('Number of days: ', int(X_test.shape[0] / period))

In [ ]:
#plots
plt.figure(figsize=(30,4))
plt.title('Train set')
plt.plot(X_train.to_numpy().flatten())
plt.show()

plt.figure(figsize=(30,4))
plt.title('Test set')
plt.plot(X_test.to_numpy().flatten())
plt.show()

We can see that in the test set we have events such as the NYC marathon, Thanksgiving, Christmas, New Years Day, and a snow storm.

### Data standardization
Standardize data by removing the mean and scaling to unit variance.

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
plt.title('Before standarize')
plt.plot(scaler.inverse_transform(X_train[0]))
plt.show()

plt.title('After standarize')
plt.plot(X_train[0])
plt.show()

### Autoencoder

In [ ]:
# params
dim_hidden1 = 32
dim_hidden2 = 16
dim_hidden3 = 8


# model
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Sequential([
          Dense(dim_hidden1, activation="relu"),
          Dense(dim_hidden2, activation="relu"),
          Dense(dim_hidden3, activation="relu")])

        self.decoder = Sequential([
          Dense(dim_hidden2, activation="relu"),
          Dense(dim_hidden1, activation="relu"),
          Dense(period, activation="sigmoid")])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

### Fitting model

In [ ]:
# params
epochs = 100
batch_size = 20
validation_split = 0.1
shuffle = False


# fitting model
autoencoder = Autoencoder()
autoencoder.compile(optimizer='adam', loss='mse')

history = autoencoder.fit(X_train, X_train, 
                          epochs = epochs, 
                          batch_size = batch_size, 
                          validation_split = validation_split, 
                          shuffle = shuffle)

### Training and Validation Loss

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.show()

### Reconstruction example on a training set

In [ ]:
encoded_data = autoencoder.encoder(X_train).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

i = 10

plt.plot(X_train[i], label='One day from train set')
plt.plot(decoded_data[i], label='Reconstruction')
plt.fill_between(np.arange(48), decoded_data[i], X_train[i], alpha=0.2, label='Error')
plt.legend()
plt.show()

print('Error (MSE): ',np.mean((X_train[i] - decoded_data[i])**2))

### Reconstruction example on a test set

In [ ]:
encoded_data = autoencoder.encoder(X_test).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

i = 10

plt.plot(X_test[i], label='One day from test set')
plt.plot(decoded_data[i], label='Reconstruction')
plt.fill_between(np.arange(48), decoded_data[i], X_test[i], alpha=0.2, label='Error')
plt.legend()
plt.show()

print('Error (MSE): ',np.mean((X_test[i] - decoded_data[i])**2))

### Loss distribution - Training set

In [ ]:
reconstruction = autoencoder.predict(X_train)
loss_train = tf.keras.losses.mae(reconstruction, X_train)

plt.title('Loss distribution - train set')
sns.distplot(loss_train, bins=50, kde=True)
plt.show()

### Loss distribution - Test set

In [ ]:
reconstruction_test = autoencoder.predict(X_test)
loss_test = tf.keras.losses.mae(reconstruction_test, X_test)

plt.title('Loss distribution - test set')
sns.distplot(loss_test, bins=50, kde=True)
plt.show()

### Selection Threshold

threshold = mean of the loss + 2 standard devation of the loss

In [ ]:
# threshold = np.quantile(loss_train, 0.95)
threshold = np.mean(loss_train) + 2*np.std(loss_train)
print(threshold)

### Loss for each day of the test set vs Threshold

In [ ]:
results = pd.DataFrame({'date': np.array(dates_test, dtype='datetime64[D]'), 
                        'loss': loss_test})

results = results.set_index('date')

results.plot(kind='bar', figsize=(30,4))
plt.axhline(threshold, color='red', label='threshold')
plt.show()

### Detected anomalies

In [ ]:
scaled_loss = (loss_test - np.min(loss_test)) / (np.max(loss_test) - np.min(loss_test)) * 0.4

In [ ]:
y_pred = loss_test.numpy() >= threshold

In [ ]:
plt.figure(figsize=(30,4))
for i in np.arange(y_pred.shape[0])[y_pred]:
    plt.axvspan(i*48, (i + 1)*48, color='r',alpha=scaled_loss[i])
plt.plot(scaler.inverse_transform(X_test).flatten())
plt.legend(['data', 'anomaly'])
plt.show()

In [ ]:
print(np.array(dates_test[y_pred]))

Most of the detected anomalies  match the following events:
* [NYC Marathon](https://en.wikipedia.org/wiki/2014_New_York_City_Marathon) (02.11.2014)
* Thanksgiving (27.11.2014)
* Christmas (24-26.12.2014)
* New Years Day (01.01.2015)
* [January 2015 North American blizzard](https://en.wikipedia.org/wiki/January_2015_North_American_blizzard) (26-27.01.2015)

![image info](https://raw.githubusercontent.com/bartk97/NYC-Taxi-Anomaly-Detection/main/Images/Data%20with%20highlighted%20anomalies.png)

### Plots of detected anomalies

In [ ]:
for d, x in zip(np.array(dates_test[y_pred]), scaler.inverse_transform(X_test)[y_pred]):
    plt.figure(figsize=(5,2))
    plt.title(d)
    plt.plot(x)
    plt.show()